In [3]:
from jqdatasdk import *
auth('15211097884','097884')
import pandas as pd
import numpy as np

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_priceNone.csv")
stock_list.head()

,ts_code,name
0,000002.SZ,万科A
1,000004.SZ,国农科技
2,000005.SZ,世纪星源
3,000006.SZ,深振业A
4,000007.SZ,全新好


In [3]:
def ts_code_trans(ts_code):
    if ts_code[7:] == "SZ" :
        return ts_code[:6] + ".XSHE"
    else:
        return ts_code[:6] + ".XSHG"
    
def query_stock(ts_code):

    q = query(
          income.statDate,
          income.code,
          indicator.roe, # 股东权益报酬率(%) RoE
          indicator.roa, # 总资产报酬率(%) RoA
          indicator.adjusted_profit, # 税后净利规模(百万)
          indicator.net_profit_margin, # 销售净利率(%)
          indicator.gross_profit_margin, # 销售毛利率(%)

          income.operating_revenue, # 营业收入
          balance.total_assets, # 总资产
          balance.total_current_assets, # 流动资产合计
          balance.total_current_liability, # 流动负债
          cash_flow.net_operate_cash_flow # 经营活动产生的现金流量净额
      ).filter(
          income.code == ts_code
      )
    
    return q

def calculate_score(rets):
    
    score = 0

    # 股东权益报酬率(%) RoE
    ROE = rets.roe.mean()
    if ROE >= 35:
        score +=550
    elif ROE >= 30:
        score +=500
    elif ROE >= 25:
        score +=450
    elif ROE >= 20:
        score +=400
    elif ROE >= 15:
        score +=300
    elif ROE >= 10:
        score +=250

    # 总资产报酬率(%) RoA
    ROA = rets.roa.mean()
    if ROA >= 15:
        score +=100
    elif ROE >= 11:
        score +=80
    elif ROE >= 7:
        score +=50

    # 税后净利规模(百万)
    adjusted_profit = rets.adjusted_profit.mean() / 1e6
    if adjusted_profit >= 10000:
        score +=150
    elif adjusted_profit >= 1000:
        score +=100

    # 现金状况分析
    operating_revenue = rets.operating_revenue.mean()
    total_assets = rets.total_assets.mean()
    total_current_assets = rets.total_current_assets.mean()
    total_assets_ = operating_revenue / total_assets
    total_current_assets_ = total_current_assets / total_assets
    if (total_current_assets_ > 0.8 and total_assets_ > 0.1) or \
        (total_current_assets < 0.8 and  total_assets_ > 0.2):
        score +=50

    # 毛利率波动(%)
    gross_profit_mean = rets.gross_profit_margin.mean()
    range_list = [ 1 if (profit-gross_profit_mean)/gross_profit_mean > 0.3 else 0 for profit in rets["gross_profit_margin"]]
    if np.array(range_list).sum() == 0:
        score += 50

    # 经营安全边际率(%)
    net_profit_margin = rets.net_profit_margin.mean()
    gross_profit_margin = rets.gross_profit_margin.mean()
    net_gross_profit = net_profit_margin / gross_profit_margin
    if net_gross_profit >= 70:
        score +=50
    elif net_gross_profit >= 50:
        score +=30
    elif net_gross_profit >= 30:
        score += 10

    def range_decay(profit, score):
        
        if profit[1]>profit[0]:
            score += 15
        else:
            score -= 15

        if profit[2]>profit[1]:
            score += 20
        else:
            score -= 20

        if profit[3]>profit[2]:
            score += 25
        else:
            score -= 25
        
        if len(profit) > 4 :
            if profit[4]>profit[3]:
                score += 30
            else:
                score -= 30

        return score

    # 经营活动现金流量
    score = range_decay(rets["net_operate_cash_flow"], score)

    # 税后净利
    score = range_decay(rets["adjusted_profit"], score)

    # 现金流量比率
    total_current_liability = rets["total_current_liability"].mean()
    net_operate_cash_flow = rets["net_operate_cash_flow"].mean()
    cash_flow_liability = net_operate_cash_flow / total_current_liability
    if cash_flow_liability >= 1.0:
        score +=50
    elif cash_flow_liability >= 0.8:
        score +=30
    
    return score

In [4]:
columns = ["stock_code", "name", "score"]
score_list = pd.DataFrame(columns = columns)

for index in range(len(stock_list)):
    
    ts_code = ts_code_trans(stock_list["ts_code"][index])
    q = query_stock(ts_code)
    
    rets = None
#     for i in range(9):
#         ret = get_fundamentals(q, statDate='201' + str(i))
#         rets = pd.concat([rets, ret], ignore_index=True) if i != 1 else ret

#     tmp_rets = rets[-5:].reset_index(drop=True) if len(rets) >= 5 else None
#     score = calculate_score(tmp_rets) if tmp_rets is not None else None
#     rets.to_csv("../data_pulled/pyramid/{}.csv".format(stock_list["ts_code"][index]), index=False)
    
    for i in range(9):
        ret = get_fundamentals(q, statDate='201' + str(i)) 
        rets = pd.concat([rets, ret], ignore_index=True) if i != 1 else ret
    
    tmp_rets = rets[-5:].reset_index(drop=True) if len(rets) == 4 else None
    score = calculate_score(tmp_rets) if tmp_rets is not None else None
    
    list_tmp = pd.DataFrame([[stock_list["ts_code"][index], stock_list["name"][index], score]], columns=columns)
    score_list = score_list.append(list_tmp)

In [5]:
score_list = score_list.sort_values(['score'],ascending=False).reset_index(drop=True)

In [6]:
score_list.to_csv("../data_pulled/stock_score_2018.csv", index=False)

In [10]:
score_list[:50]

,stock_code,name,score
0,601155.SH,新城控股,760
1,603866.SH,桃李面包,720
2,603899.SH,晨光文具,670
3,002749.SZ,国光股份,670
4,603568.SH,伟明环保,670
5,001979.SZ,招商蛇口,670
6,603898.SH,好莱客,650
7,603589.SH,口子窖,620
8,601985.SH,中国核电,600
9,002777.SZ,久远银海,570
